# Extracting datasets

## Relevant Imports

In [1]:
%matplotlib inline

from datetime import datetime

import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio

from adjustText import adjust_text
from shapely.geometry import Point, box
from rasterio.mask import mask
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## VIIRS Data

### Japan Data

In [7]:
# Filepaths
import os
boundaries_folder = r"./boundaires/"


boundaries_folder = "./boundaires/"  # Use the exact folder name from your screenshot
raster_file = r'./SVDNB_npp_20230101-20230131_75N060E_vcmcfg_v10_c202302080600.avg_rade9h.tif'
boundary_file = os.path.join(boundaries_folder, "geoBoundaries-JPN-ADM0.geojson")
output_csv = r'Japan_light_intensity.csv'
try:
    with rasterio.open(raster_file) as src:
        print(src.profile)
except rasterio.errors.RasterioIOError:
    print(f"File not found: {raster_file}")



# Load Japan's Boundary
Japan = gpd.read_file(boundary_file)
Japan = Japan.to_crs(epsg=4326)

print(Japan.total_bounds)
# Open the Raster and Check Overlap
with rasterio.open(raster_file) as src:
    raster_bounds = box(*src.bounds)
    print("Raster Bounds:", src.bounds)
    print("Japan Bounds:", Japan.total_bounds)

    if not raster_bounds.intersects(Japan.unary_union):
        raise ValueError("Japan's boundary does not overlap with the raster extent.")

    # Clip the raster
    Japan_geom_list = [feature["geometry"] for feature in Japan.__geo_interface__["features"]]
    clipped_raster, clipped_transform = mask(src, Japan_geom_list, crop=True)

# Extract Raster Values
light_intensity = clipped_raster[0]
rows, cols = np.where(~np.isnan(light_intensity))
values = light_intensity[rows, cols]
x_coords, y_coords = rasterio.transform.xy(clipped_transform, rows, cols)

data = pd.DataFrame({
    'longitude': x_coords,
    'latitude': y_coords,
    'light_intensity': values
})
data.to_csv(output_csv, index=False)
print(f"Extracted data saved to {output_csv}")


File not found: ./SVDNB_npp_20230101-20230131_75N060E_vcmcfg_v10_c202302080600.avg_rade9h.tif


DataSourceError: ./boundaires/geoBoundaries-JPN-ADM0.geojson: No such file or directory

### Philippines Data

In [3]:
# Filepaths
import os
boundaries_folder = r"./boundaires/" 
raster_file = r'./SVDNB_npp_20230101-20230131_75N060E_vcmcfg_v10_c202302080600.avg_rade9h.tif'
boundary_file=os.path.join(boundaries_folder, "geoBoundaries-PHL-ADM0.geojson")
output_csv = r'Philippines_light_intensity.csv'

# Load Taiwan's Boundary
Philippines = gpd.read_file(boundary_file)
Philippines = Philippines.to_crs(epsg=4326)

print(Philippines.total_bounds)
# Open the Raster and Check Overlap
with rasterio.open(raster_file) as src:
    raster_bounds = box(*src.bounds)
    print("Raster Bounds:", src.bounds)
    print("Philippines Bounds:", Philippines.total_bounds)

    if not raster_bounds.intersects(Philippines.unary_union):
        raise ValueError("Philippines's boundary does not overlap with the raster extent.")

    # Clip the raster
    Philippines_geom_list = [feature["geometry"] for feature in Philippines.__geo_interface__["features"]]
    clipped_raster, clipped_transform = mask(src, Philippines_geom_list, crop=True)

# Extract Raster Values
light_intensity = clipped_raster[0]
rows, cols = np.where(~np.isnan(light_intensity))
values = light_intensity[rows, cols]
x_coords, y_coords = rasterio.transform.xy(clipped_transform, rows, cols)

data = pd.DataFrame({
    'longitude': x_coords,
    'latitude': y_coords,
    'light_intensity': values
})
data.to_csv(output_csv, index=False)
print(f"Extracted data saved to {output_csv}")


DataSourceError: ./boundaires/geoBoundaries-PHL-ADM0.geojson: No such file or directory

### Taiwan Data 

In [4]:
# Filepaths
import os
boundaries_folder = "./boundaires/" 
raster_file = r'./SVDNB_npp_20230101-20230131_75N060E_vcmcfg_v10_c202302080600.avg_rade9h.tif'
boundary_file=os.path.join(boundaries_folder, "geoBoundaries-TWN-ADM0.geojson")

output_csv = r'Taiwan_light_intensity.csv'

# Load Taiwan's Boundary
Taiwan = gpd.read_file(boundary_file)
Taiwan = Taiwan.to_crs(epsg=4326)

print(Taiwan.total_bounds)
# Open the Raster and Check Overlap
with rasterio.open(raster_file) as src:
    raster_bounds = box(*src.bounds)
    print("Raster Bounds:", src.bounds)
    print("Taiwan Bounds:", Taiwan.total_bounds)

    if not raster_bounds.intersects(Taiwan.unary_union):
        raise ValueError("Taiwan's boundary does not overlap with the raster extent.")

    # Clip the raster
    Taiwan_geom_list = [feature["geometry"] for feature in Taiwan.__geo_interface__["features"]]
    clipped_raster, clipped_transform = mask(src, Taiwan_geom_list, crop=True)

# Extract Raster Values
light_intensity = clipped_raster[0]
rows, cols = np.where(~np.isnan(light_intensity))
values = light_intensity[rows, cols]
x_coords, y_coords = rasterio.transform.xy(clipped_transform, rows, cols)

data = pd.DataFrame({
    'longitude': x_coords,
    'latitude': y_coords,
    'light_intensity': values
})
data.to_csv(output_csv, index=False)
print(f"Extracted data saved to {output_csv}")


[118.20920337  21.89259297 122.03704767  26.25789185]
Raster Bounds: BoundingBox(left=59.99791666665, bottom=0.0020827333499937595, right=179.99791762665, top=75.00208333335)
Taiwan Bounds: [118.20920337  21.89259297 122.03704767  26.25789185]


C:\Users\seigk\AppData\Local\Temp\ipykernel_14812\2245409757.py:20: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  if not raster_bounds.intersects(Taiwan.unary_union):


Extracted data saved to Taiwan_light_intensity.csv


## Ookla Speedtest Data

In [5]:
%matplotlib inline

from datetime import datetime
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely.geometry import Point
from adjustText import adjust_text

In [6]:
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


In [7]:
def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)
    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

In [9]:
import requests
import zipfile
import os

# URL of the shapefile
tile_url = get_tile_url("fixed", 2023, 2)

# Download the zip file
response = requests.get(tile_url)
if response.status_code == 200:
    with open("performance_tiles.zip", 'wb') as f:
        f.write(response.content)
    print("Download complete.")

    # Unzip the file
    with zipfile.ZipFile("performance_tiles.zip", 'r') as zip_ref:
        zip_ref.extractall("performance_tiles")
    print("File extracted.")
    
    # Read shapefile
    shapefile_path = "performance_tiles/your_shapefile.shp"  # Adjust the path to the shapefile
    tiles = gpd.read_file(shapefile_path)
    print(tiles.head())
else:
    print(f"Failed to download the file, status code: {response.status_code}")


In [1]:
import os

# Define the relative path to the boundaries folder
boundaries_folder = "./boundaires/" 

# Dictionary with country names and corresponding file names
geojson_files = {
    "Philippines": os.path.join(boundaries_folder, "geoBoundaries-PHL-ADM0.geojson"),
    "Japan": os.path.join(boundaries_folder, "geoBoundaries-JPN-ADM0.geojson"),
    "Taiwan": os.path.join(boundaries_folder, "geoBoundaries-TWN-ADM0.geojson"),
}

# Initialize an empty list to store results
all_tiles = []

# Loop through each country and process the tiles
for country, geojson_path in geojson_files.items():
    # Load the boundary from GeoJSON file
    boundary = gpd.read_file(geojson_path)
    boundary = boundary.to_crs(4326)  # Ensure CRS matches the tiles
    print(f"{country} boundary loaded:")
    print(boundary.head())
    
    
    # Perform spatial join between tiles and the country's boundary
    country_tiles = gpd.sjoin(tiles, boundary, how="inner", predicate='intersects')

    
    # Convert speeds to Mbps
    country_tiles['avg_d_mbps'] = country_tiles['avg_d_kbps'] / 1000
    country_tiles['avg_u_mbps'] = country_tiles['avg_u_kbps'] / 1000
    country_tiles['country'] = country  # Add a column to distinguish countries
    
    # Append to the list
    all_tiles.append(country_tiles)

# Combine all results into a single GeoDataFrame
combined_tiles = gpd.GeoDataFrame(pd.concat(all_tiles, ignore_index=True))
print("Combined tiles data:")
print(combined_tiles.head())

# save as csv file 
combined_tiles.to_csv("combined_tiles.csv", index=False)


NameError: name 'gpd' is not defined

## Data preocessing


## extract radiance data and normalized

radiance_data['normalized_radiance'] = (radiance_data['radiance'] - radiance_data['radiance'].min()) / (
    radiance_data['radiance'].max() - radiance_data['radiance'].min()
)

print("Radiance data extracted and normalized.")
radiance_data.to_csv("normalized_radiance_data.csv", index=False)

## Aggregate Internet Performance Metrics and compute average internet speed and latency

In [ ]:
# Perform Spatial Join with City Boundaries
ookla_city_join = gpd.sjoin(country_tiles, boundary, how="inner", predicate="intersects")

# Compute Averages for Each City
aggregated_metrics = ookla_city_join.groupby('city_name').agg({
    'avg_d_kbps': 'mean',  # Average download speed
    'avg_u_kbps': 'mean',  # Average upload speed
    'avg_latency_ms': 'mean'  # Average latency
}).reset_index()



print("Internet performance metrics aggregated.")
aggregated_metrics.to_csv("aggregated_internet_metrics.csv", index=False)

## integration of radiance data and internet performance metrics

In [ ]:
import geopandas as gpd
import pandas as pd

# Filepaths for the radiance data and Ookla data
radiance_file = "normalized_radiance_data.csv"
ookla_tiles_file = "combined_tiles.csv"

# Load datasets
radiance_data = pd.read_csv(radiance_file)
ookla_tiles = gpd.read_file(ookla_tiles_file)

# Convert radiance data to GeoDataFrame
radiance_gdf = gpd.GeoDataFrame(
    radiance_data,
    geometry=gpd.points_from_xy(radiance_data.longitude, radiance_data.latitude),
    crs="EPSG:4326"
)

# Ensure Ookla tiles are in the same CRS
ookla_tiles = ookla_tiles.to_crs("EPSG:4326")

# Perform spatial join: radiance data -> Ookla tiles
integrated_data = gpd.sjoin(radiance_gdf, ookla_tiles, how="inner", predicate="intersects")

# Combine data: keep relevant columns
integrated_data = integrated_data[[
    "longitude", "latitude", "normalized_radiance", 
    "avg_d_mbps", "avg_u_mbps", "avg_lat_ms", "country"
]]

# Save the integrated data to a CSV file
output_file = "integrated_data.csv"
integrated_data.to_csv(output_file, index=False)
print(f"Integrated data saved to {output_file}")


## visualization

## Heatmap: Radiance Intensity and Internet Speeds

In [ ]:
# Heatmap for Radiance Intensity and Internet Speeds
plt.figure(figsize=(10, 8))
heatmap_data = combined_tiles.pivot_table(index='latitude', columns='longitude', values='normalized_radiance', aggfunc='mean')
sns.heatmap(heatmap_data, cmap='viridis', cbar_kws={'label': 'Normalized Radiance'})
plt.title('Heatmap of Radiance Intensity')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

plt.figure(figsize=(10, 8))
heatmap_data_speed = combined_tiles.pivot_table(index='latitude', columns='longitude', values='avg_d_mbps', aggfunc='mean')
sns.heatmap(heatmap_data_speed, cmap='plasma', cbar_kws={'label': 'Average Download Speed (Mbps)'})
plt.title('Heatmap of Internet Speeds')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


## Scatter Plot: Radiance vs. Internet Performance

In [ ]:
# Scatter Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=combined_tiles, x='normalized_radiance', y='avg_d_mbps', hue='country', alpha=0.7)
plt.title('Radiance vs Internet Performance')
plt.xlabel('Normalized Radiance')
plt.ylabel('Average Download Speed (Mbps)')
plt.legend(title='Country', loc='upper left')
plt.show()


## Bar Chart: Internet Performance Across Urban Areas

In [ ]:
# Bar Chart
urban_areas = combined_tiles.groupby('urban_region').agg({'avg_d_mbps': 'mean', 'normalized_radiance': 'mean'}).reset_index()
urban_areas = urban_areas.sort_values('normalized_radiance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(data=urban_areas, x='urban_region', y='avg_d_mbps', palette='coolwarm')
plt.title('Internet Performance Across Urban Areas')
plt.xlabel('Urban Region')
plt.ylabel('Average Download Speed (Mbps)')
plt.xticks(rotation=45, ha='right')
plt.show()


Geospatial Overlay: Internet Speeds and Radiance

In [ ]:
# Geospatial Overlay
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
combined_tiles.plot(column='avg_d_mbps', cmap='coolwarm', legend=True, ax=ax, legend_kwds={'label': "Average Download Speed (Mbps)"})
combined_tiles.plot(column='normalized_radiance', cmap='viridis', alpha=0.5, ax=ax, legend=False)
plt.title('Geospatial Overlay of Internet Speeds and Radiance')
plt.show()


Line Chart: Trends Over Time

In [ ]:
# Line Chart for Trends Over Time
if 'time' in combined_tiles.columns:
    time_trends = combined_tiles.groupby('time').agg({'avg_d_mbps': 'mean', 'normalized_radiance': 'mean'}).reset_index()

    plt.figure(figsize=(12, 6))
    plt.plot(time_trends['time'], time_trends['avg_d_mbps'], label='Internet Speed (Mbps)', marker='o')
    plt.plot(time_trends['time'], time_trends['normalized_radiance'], label='Radiance', marker='o', linestyle='--')
    plt.title('Trends in Internet Speed and Radiance Over Time')
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()
else:
    print("Time column not found. Please ensure your data contains a 'time' column for trend analysis.")


## Hypothesis Testing:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Data preprocessing for hypothesis testing
data_for_testing = combined_tiles[['avg_d_mbps', 'normalized_radiance']]

# Drop NaN values
data_for_testing.dropna(subset=['avg_d_mbps', 'normalized_radiance'], inplace=True)

# Scatter plot to visualize correlation
plt.figure(figsize=(10, 6))
plt.scatter(data_for_testing['normalized_radiance'], data_for_testing['avg_d_mbps'], alpha=0.5)
plt.title('Correlation between Internet Performance and Nighttime Light Intensity')
plt.xlabel('Normalized Radiance')
plt.ylabel('Average Download Speed (Mbps)')
plt.grid(True)
plt.show()

# Perform Pearson correlation test
corr_coefficient, p_value = pearsonr(data_for_testing['normalized_radiance'], data_for_testing['avg_d_mbps'])

# Print the results
print(f"Pearson Correlation Coefficient: {corr_coefficient}")
print(f"P-value: {p_value}")

# Hypothesis testing
alpha = 0.05  # significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant correlation between internet performance and nighttime light intensity.")
else:
    print("Fail to reject the null hypothesis: No significant correlation between internet performance and nighttime light intensity.")


## •	Predict internet speeds in urban regions based on nighttime light intensity.

In [ ]:


# Preprocessing the data
data_for_modeling = combined_tiles[['avg_d_mbps', 'normalized_radiance']]
data_for_modeling.dropna(subset=['avg_d_mbps', 'normalized_radiance'], inplace=True)

# Split into features and target
X = data_for_modeling[['normalized_radiance']]
y = data_for_modeling['avg_d_mbps']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model building with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
print(f"R-squared: {r2_score(y_test, y_pred)}")

# Visualization
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.title('Actual vs Predicted Internet Speeds')
plt.xlabel('Actual Internet Speed (Mbps)')
plt.ylabel('Predicted Internet Speed (Mbps)')
plt.show()

# Making new predictions
new_radiance = pd.DataFrame({'normalized_radiance': [0.6]})  # Example input
predicted_speed = model.predict(new_radiance)
print(f"Predicted Internet Speed (Mbps) for normalized radiance of 0.6: {predicted_speed[0]}")
